Local area migration indicators UK: Migration Flows

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'migrationwithintheuk/datasets/localareamigrationindicatorsunitedkingdom')
scraper

## Local area migration indicators, UK

Spreadsheet bringing together different migration-related data sources at local authority level.

### Distributions

1. Local area migration indicators, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/migrationwithintheuk/datasets/localareamigrationindicatorsunitedkingdom/current/publicviewmastercopylocalareamigrationindicatoraugust2018.xlsx))


In [2]:
sheets = scraper.distribution().as_databaker()
tab = [t for t in sheets if t.name == 'Migration Flows'][0]

In [3]:
MigrationArea = tab.filter('Area Code').fill(DOWN).is_not_blank().is_not_whitespace()
Area = tab.filter('Area Name').fill(DOWN).is_not_blank().is_not_whitespace()
Period = tab.filter(contains_string('Mid-')).is_not_number() - tab.filter(contains_string('Population'))
migration = tab.filter(contains_string('Migration')).is_not_number()
migration = HDim(migration,'Migration',CLOSEST,LEFT)
migration.AddCellValueOverride('Internal Migration\n (within UK)', 'Internal Migration (within UK)')
flow = tab.filter(contains_string('Inflow')) | tab.filter(contains_string('Outflow'))
obs = tab.same_row(MigrationArea) - Area - MigrationArea 
obs = obs - tab.filter(contains_string('Population')).fill(DOWN)

In [4]:
Dimensions = [
            HDim(Period,'Mid Year',CLOSEST,LEFT),
            HDim(MigrationArea,'Geography', DIRECTLY, LEFT),
            migration,
            HDim(flow,'Flow',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]
c1 = ConversionSegment(obs, Dimensions, processTIMEUNIT=True)
savepreviewhtml(c1)

In [5]:
tidy = c1.topandas()

We can drop the mid year population estimates, as these form a separate dataset.

In [6]:
tidy = tidy[~tidy['Migration'].str.match('^Mid-[0-9]{4} Population Estimate$')].copy()

In [7]:
tidy['OBS'].replace('', pd.np.nan, inplace=True)
tidy.dropna(subset=['OBS'], inplace=True)
tidy['Value'] = tidy['OBS'].astype(int)

In [8]:
tidy['Mid Year'] = tidy['Mid Year'].str.replace(r'^Mid-([0-9]{4}) to Mid-([0-9]{4})\s*$',
                                                      lambda m: f"{m.group(1)}-06-30T00:00:00/P1Y")

In [9]:
tidy = tidy[['Mid Year','Geography','Migration','Flow','Measure Type','Value','Unit']]

In [10]:
from pathlib import Path
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

tidy.to_csv(destinationFolder / ('migrationflows.csv'), index = False)

In [11]:
scraper.dataset.family = 'migration'
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [12]:
tidy

,Mid Year,Geography,Migration,Flow,Measure Type,Value,Unit
0,2007-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Inflow,Count,553735,People
1,2007-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Outflow,Count,316545,People
2,2007-06-30T00:00:00/P1Y,E92000001,Internal Migration (within UK),Inflow,Count,98843,People
3,2007-06-30T00:00:00/P1Y,E92000001,Internal Migration (within UK),Outflow,Count,116591,People
4,2008-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Inflow,Count,525074,People
5,2008-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Outflow,Count,350353,People
6,2008-06-30T00:00:00/P1Y,E92000001,Internal Migration (within UK),Inflow,Count,99548,People
7,2008-06-30T00:00:00/P1Y,E92000001,Internal Migration (within UK),Outflow,Count,104650,People
8,2009-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Inflow,Count,514257,People
9,2009-06-30T00:00:00/P1Y,E92000001,Long-Term International Migration,Outflow,Count,294075,People
